# PREAMBLE

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import os
import matplotlib.pyplot as plt
import pyximport
import pandas as pd
import numpy as np
import gzip
import pickle
from datetime import datetime as dt
codebase_path = os.path.abspath('..')
sys.path.append(codebase_path)
sys.path.append('.')

from data_utils import *
pyximport.install()
from helpers.load import load_data, load_pickle  # noqa: E402
from helpers.save import save  # noqa: E402
from definitions.path import data_interim, data_processed  # noqa: E402
raw_df_path = os.path.join(data_interim, 'patient_timeseries_small.csv')


# LOAD PHYSIO & OBSERVATION DATA

In [2]:
df_raw = load_data('patient_timeseries_small.p', data_interim)

In [4]:
with open ('./data/obs_grps.pkl', 'rb') as f:
    obs_df = pickle.load(f)

# THE DATA PROCESSING PROCEDURE
STARTING FROM RAW PHYSIO MARKES AND ENDING WITH MODEL TRAINING FEATURE ARRAYS

#### 1. LOAD DATA PROCESSING CLASSES

In [6]:
processor = process_raw() # PROCESSING RAW PHYSIO DATA
constructor = construct_features() # ENGINEER STATISTICAL FEATURES

#### 2. SEPARATE THE CASE AND CONTOL PATIENTS AND THE SUBSET OF EFFECTIVE PHSYIO FEATURES
INPUTS ARE THE RAW TIMESERIES PATIENT DATA AND THE OBSERVATIONS DATA.

In [ ]:
case_df, control_df = processor.ctrl_test_dfs(df_raw, obs_df)  # MEMORY INTENSTIVE00

In [5]:
case_df = pd.read_csv('./data/case_df.csv', index_col= 0)
control_df = pd.read_csv('./data/control_df.csv', index_col= 0)
control_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11335492 entries, 1678 to 3395784
Data columns (total 13 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   measurement_datetime                  float64
 1   hours_since_birth                     float64
 2   Monitor Arteriele Bloeddruk Diastole  float64
 3   event                                 object 
 4   Monitor Arteriele Bloeddruk Mean      float64
 5   Monitor Arteriele Bloeddruk Systole   float64
 6   Monitor Hartfrequentie                float64
 7   Monitor Ademhalingsfrequentie         float64
 8   Monitor O2 Saturatie                  float64
 9   Couveuse Gemeten Temp                 float64
 10  Monitor Temperatuur 1                 float64
 11  Monitor Hartfrequentie Pulse          float64
 12  Monitor Hartfrequentie Pleth          float64
dtypes: float64(12), object(1)
memory usage: 1.2+ GB


In [129]:
case_df.shape , control_df.shape

((14057511, 13), (11335492, 13))

#### 3. EXTRACT THE HORIZON SEGMENT PER CASE AND CONTROL.
Extracts the 12hrs preceeding `t_sepsis` and `t_control` per patient.

In [7]:
case_horizon, control_horizon = processor.subset_horizon(case_df, control_df)

#### Note: All case patients' timeseries ends with a postive bloodculture event (aka. t_sepsis). Ex: patient 4611

In [11]:
case_horizon.loc[4611] 

,measurement_datetime,hours_since_birth,Monitor Arteriele Bloeddruk Diastole,event,Monitor Arteriele Bloeddruk Mean,Monitor Arteriele Bloeddruk Systole,Monitor Hartfrequentie,Monitor Ademhalingsfrequentie,Monitor O2 Saturatie,Couveuse Gemeten Temp,Monitor Temperatuur 1,Monitor Hartfrequentie Pulse,Monitor Hartfrequentie Pleth
patient_id,,,,,,,,,,,,,
4611,3870.0,64.500000,NaN,NaN,NaN,NaN,155.0,83.0,95.400002,30.000000,37.450001,NaN,NaN
4611,3871.0,64.516667,NaN,NaN,NaN,NaN,157.0,89.0,94.000000,30.000000,37.450001,NaN,NaN
4611,3872.0,64.533333,NaN,NaN,NaN,NaN,158.0,87.0,94.300003,30.000000,37.450001,NaN,NaN
4611,3873.0,64.550000,NaN,NaN,NaN,NaN,158.0,85.0,93.599998,30.000000,37.459999,NaN,NaN
4611,3874.0,64.566667,NaN,NaN,NaN,NaN,159.0,77.0,92.900002,30.000000,37.450001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,4586.0,76.433333,NaN,NaN,NaN,NaN,174.0,50.0,99.400002,29.299999,36.290001,NaN,NaN
4611,4587.0,76.450000,NaN,NaN,NaN,NaN,162.0,63.0,99.599998,29.400000,36.279999,NaN,NaN
4611,4588.0,76.466667,NaN,NaN,NaN,NaN,177.0,44.0,99.699997,29.500000,36.290001,NaN,NaN


#### 4. CONSTRUCTS STATISTICAL FEATURE VECTOR PER PATIENT.
SUMMARIZE EACH PATIENT'S RECORD BY AGGREGATING 8 STATISTICAL FEATURES THROUHG A SLIDING WINDOW OF 3HRS.<br>
Each patient has 403 features.

In [13]:
case_features = constructor.feature_df(case_horizon, obs_df)
case_features

Int_0_kurt                                \
           Couveuse Gemeten Temp Monitor Ademhalingsfrequentie   
patient_id                                                       
4611                   33.272855                      1.380111   
11621                  23.052150                      0.114882   
23590                  -1.638770                     -0.317602   
26695                  22.415269                      0.288914   
29830                   7.257185                     -0.730481   
...                          ...                           ...   
3370016                -0.818942                     -0.153506   
3371533                 3.129030                     -0.453941   
3384445                      NaN                     -0.160036   
3385344                12.618330                     -0.830147   
3392486                10.138659                     -0.948057   

                                                 \
           Monitor Arteriele Bloeddruk Diastole   
patient_id                                        
4611                                        NaN   
11621                                       NaN   
23590                                       NaN   
26695                                       NaN   
29830                                       NaN   
...                                         ...   
3370016                                3.029857   
3371533                                     NaN   
3384445                                     NaN   
3385344                               -0.089005   
3392486                                     NaN   

                                             \
           Monitor Arteriele Bloeddruk Mean   
patient_id                                    
4611                                    NaN   
11621                                   NaN   
23590                                   NaN   
26695                                   NaN   
29830                                   NaN   
...                                     ...   
3370016                            3.146477   
3371533                                 NaN   
3384445                                 NaN   
3385344                           -0.253868   
3392486                                 NaN   

                                                                       \
           Monitor Arteriele Bloeddruk Systole Monitor Hartfrequentie   
patient_id                                                              
4611                                       NaN              -0.957576   
11621                                      NaN              13.004031   
23590                                      NaN              29.289697   
26695                                      NaN               3.860096   
29830                                      NaN              11.076638   
...                                        ...                    ...   
3370016                               3.109573               0.599118   
3371533                                    NaN              -0.191043   
3384445                                    NaN               1.721785   
3385344                               0.286699              -0.206766   
3392486                                    NaN               6.295299   

                                                                      \
           Monitor Hartfrequentie Pleth Monitor Hartfrequentie Pulse   
patient_id                                                             
4611                                NaN                          NaN   
11621                               NaN                          NaN   
23590                               NaN                          NaN   
26695                               NaN                          NaN   
29830                               NaN                          NaN   
...                                 ...                          ...   
3370016                        0.282275             

In [14]:
control_features = constructor.feature_df(control_horizon, obs_df)
control_features

Int_0_kurt                                \
           Couveuse Gemeten Temp Monitor Ademhalingsfrequentie   
patient_id                                                       
1678                    5.451859                     -0.963789   
1809                    6.104629                           NaN   
14404                  11.654846                           NaN   
24188                   0.042218                     -0.437840   
34870                  33.857598                     -1.022441   
...                          ...                           ...   
3363458                 3.187841                     -1.069018   
3372236                 6.572519                      0.821376   
3377022                 4.281828                     -0.899166   
3393381                10.283293                     -0.743255   
3395784               180.000000                     -0.354904   

                                                 \
           Monitor Arteriele Bloeddruk Diastole   
patient_id                                        
1678                                   0.513203   
1809                                   0.036746   
14404                                       NaN   
24188                                       NaN   
34870                                  3.797997   
...                                         ...   
3363458                               -0.206113   
3372236                                0.867421   
3377022                               54.173907   
3393381                                     NaN   
3395784                                     NaN   

                                             \
           Monitor Arteriele Bloeddruk Mean   
patient_id                                    
1678                               0.996446   
1809                               0.052493   
14404                                   NaN   
24188                                   NaN   
34870                              3.320847   
...                                     ...   
3363458                           -0.513845   
3372236                            0.008326   
3377022                           57.155478   
3393381                                 NaN   
3395784                                 NaN   

                                                                       \
           Monitor Arteriele Bloeddruk Systole Monitor Hartfrequentie   
patient_id                                                              
1678                                  0.481717              18.148413   
1809                                  0.049107                    NaN   
14404                                      NaN                    NaN   
24188                                      NaN              -0.187679   
34870                                 0.525998               1.697341   
...                                        ...                    ...   
3363458                              -0.444289               2.157987   
3372236                              -0.049627               2.320742   
3377022                              60.256593               1.881823   
3393381                                    NaN               3.327318   
3395784                                    NaN               0.368981   

                                                                      \
           Monitor Hartfrequentie Pleth Monitor Hartfrequentie Pulse   
patient_id                                                             
1678                                NaN                          NaN   
1809                                NaN                          NaN   
14404                               NaN                          NaN   
24188                               NaN                          NaN   
34870                               NaN                          NaN   
...                                 ...                          ...   
3363458                        2.387060             

In [14]:
control_features.columns.to_list()

[('Int_0_kurt', 'Couveuse Gemeten Temp'),
 ('Int_0_kurt', 'Monitor Ademhalingsfrequentie'),
 ('Int_0_kurt', 'Monitor Arteriele Bloeddruk Diastole'),
 ('Int_0_kurt', 'Monitor Arteriele Bloeddruk Mean'),
 ('Int_0_kurt', 'Monitor Arteriele Bloeddruk Systole'),
 ('Int_0_kurt', 'Monitor Hartfrequentie'),
 ('Int_0_kurt', 'Monitor Hartfrequentie Pleth'),
 ('Int_0_kurt', 'Monitor Hartfrequentie Pulse'),
 ('Int_0_kurt', 'Monitor O2 Saturatie'),
 ('Int_0_kurt', 'Monitor Temperatuur 1'),
 ('Int_0_max', 'Couveuse Gemeten Temp'),
 ('Int_0_max', 'Monitor Ademhalingsfrequentie'),
 ('Int_0_max', 'Monitor Arteriele Bloeddruk Diastole'),
 ('Int_0_max', 'Monitor Arteriele Bloeddruk Mean'),
 ('Int_0_max', 'Monitor Arteriele Bloeddruk Systole'),
 ('Int_0_max', 'Monitor Hartfrequentie'),
 ('Int_0_max', 'Monitor Hartfrequentie Pleth'),
 ('Int_0_max', 'Monitor Hartfrequentie Pulse'),
 ('Int_0_max', 'Monitor O2 Saturatie'),
 ('Int_0_max', 'Monitor Temperatuur 1'),
 ('Int_0_mean', 'Couveuse Gemeten Temp'),
 ('I

#### 5. CONSTRUCT THE FINAL MODELING DATA WITH LABELS

In [17]:
modeling_set = constructor.modeling_set(case_features, control_features)
modeling_set

Int_0_kurt                                \
           Couveuse Gemeten Temp Monitor Ademhalingsfrequentie   
patient_id                                                       
3000905                13.220986                     -0.695071   
1374974                -1.742137                           NaN   
3072442                41.191212                     -0.259215   
3049134                 2.657251                     11.620385   
702346                 -0.837810                           NaN   
...                          ...                           ...   
3109678                 4.713343                     -0.490114   
1858350                -0.516409                     -0.508673   
2924962                 0.868535                     -0.179619   
2960446                      NaN                      3.187489   
2959905                21.377105                      2.057059   

                                                 \
           Monitor Arteriele Bloeddruk Diastole   
patient_id                                        
3000905                                     NaN   
1374974                                     NaN   
3072442                                     NaN   
3049134                                     NaN   
702346                                      NaN   
...                                         ...   
3109678                                     NaN   
1858350                               -0.301509   
2924962                                     NaN   
2960446                                7.976470   
2959905                                1.362404   

                                             \
           Monitor Arteriele Bloeddruk Mean   
patient_id                                    
3000905                                 NaN   
1374974                                 NaN   
3072442                                 NaN   
3049134                                 NaN   
702346                                  NaN   
...                                     ...   
3109678                                 NaN   
1858350                           -0.656022   
2924962                                 NaN   
2960446                            9.607011   
2959905                            8.200481   

                                                                       \
           Monitor Arteriele Bloeddruk Systole Monitor Hartfrequentie   
patient_id                                                              
3000905                                    NaN               2.560210   
1374974                                    NaN                    NaN   
3072442                                    NaN               0.489788   
3049134                                    NaN              -0.277844   
702346                                     NaN                    NaN   
...                                        ...                    ...   
3109678                                    NaN              43.287897   
1858350                              -0.194166              -0.148049   
2924962                                    NaN              10.162363   
2960446                               3.283504              -0.518386   
2959905                              -0.433994               2.634327   

                                                                      \
           Monitor Hartfrequentie Pleth Monitor Hartfrequentie Pulse   
patient_id                                                             
3000905                        2.863728                          NaN   
1374974                             NaN                          NaN   
3072442                        0.524500                          NaN   
3049134                       -0.275088                          NaN   
702346                              NaN                          NaN   
...                                 ...                          ...   
3109678                       11.343593             

#### 6. ARRAYS FOR MODEL DEVELOPMENT

In [19]:
X, y = constructor.training_arrays(modeling_set)
X.shape, y.shape

((792, 403), (792,))